# Analytics for Unstructured Data (F2025) - Assignment 2
## Building a Crowdsourced Recommender System


**Team Members:** Christian Breton, Mohar Chaudhuri, Stiles Clements, Muskan Khepar, Franco Salinas, Rohini Sondole

**Due Date:** September 26th, 2025 by 11:59 PM  
**Dataset:** Beer Reviews from BeerAdvocate.com  

---

## **Assignment Overview**

This assignment focuses on building a comprehensive crowdsourced recommender system using beer reviews scraped from BeerAdvocate.com. We will implement and compare different recommendation approaches including:

- **Bag-of-words** with cosine similarity
- **Pre-trained word embeddings** (spaCy)
- **Custom word embeddings** trained on our data
- **Sentiment analysis** for attribute scoring
- **Similarity-based competitor analysis**

## **Table of Contents**

- **Task A:** Web Scraping - Extract 8-10k beer reviews (~250 beers)
- **Task B:** Bag-of-words Recommender with sentiment analysis
- **Task C:** Word embeddings comparison (spaCy vs bag-of-words)
- **Task D:** Custom word embeddings implementation
- **Task E:** Rating-based vs. attribute-based recommendations analysis
- **Task F:** Product similarity and competitor identification


---

In [4]:
import pandas as pd #tabular data handling

import re 
from collections import Counter, defaultdict
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import numpy as np
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from collections import defaultdict
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [6]:
from functools import reduce
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer


# Task A: Web Scraping Implementation Comments


In [ ]:
#Here I import libraries
import time #this one handles delays and pauses
import random #generates random numbers I used to mimmic human behavior
import pandas as pd #tabular data handling

from selenium import webdriver #automates browser beer review loading and interacts w elements
from selenium.webdriver.common.by import By #handles locator strategies for elements 
from selenium.webdriver.support.ui import WebDriverWait #sets waits for elements to load
from selenium.webdriver.support import expected_conditions as EC #sets conditions in WebDriverWait for specific elements
import requests #sends HTTP requests to fetch static HTML (used for getting the list of top beers)
from bs4 import BeautifulSoup #parses HTML to extract beer names and URLs from beer advocate
import os #handles checking if output CSV exists and saves HTML files

# -----------------------------
# Step 1: Get top beers
# -----------------------------
def get_top_beer_urls():
    '''Sends a request to BeerAdvocate Top 250 page with a
    browser-like user agent. It returns an error if the request
    fails. It uses BeautifulSoup to parse HTML. Inside each sell it searched for
    the links whose href has 5 slashes.
    Extracts the beer name and builds the beer URL'''
    url = 'https://www.beeradvocate.com/beer/top-rated/'
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to fetch top beers: {e}")
        return []

    soup = BeautifulSoup(resp.text, 'html.parser')
    beer_data = []

    for td in soup.select('td.hr_bottom_light'):
        try:
            beer_a_tag = td.find('a', href=lambda href: href and href.count('/') == 5)
            if beer_a_tag:
                beer_name = beer_a_tag.get_text().strip()
                beer_url = 'https://www.beeradvocate.com' + beer_a_tag['href']
                beer_data.append({'name': beer_name, 'url': beer_url})
        except:
            continue

    return beer_data

# -----------------------------
# Human-like behavior functions
# -----------------------------
def human_delay(base_range=(2, 5)):
    '''Makes the scraper behave more like a human.
       Randomly selects number that defines the duration of a pause'''
    delay = random.uniform(*base_range)
    print(f"⏳ Sleeping {delay:.1f}s")
    time.sleep(delay)

def slow_scroll(driver, steps_range=(8, 15), pause_range=(0.7, 1.5)):
    '''Choose a random number of scroll steps between the range, and for each
        picks a random vertical scroll distance employing selenium to scroll the browser
        and pauses for a random short duration'''
    steps = random.randint(*steps_range)
    for _ in range(steps):
        scroll_amount = random.randint(50, 200)
        driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
        time.sleep(random.uniform(*pause_range))

def long_break_after_beers(driver, beer_index):
    '''After checking if the current beer index is a multiple of 4 
       it picks a random break time of between 3 and 5 minutes.
       It calls the scrolling function while on break and after 
       the break is over calls ensure_logged_in to verify we are
       logged_in'''
    if (beer_index + 1) % 4 == 0:  # every 4 beers
        long_wait = random.uniform(180, 300)  # 3–5 minutes
        print(f"😴 Taking a long break after {beer_index+1} beers: {long_wait/60:.1f} minutes")
        end_time = time.time() + long_wait
        while time.time() < end_time:
            slow_scroll(driver)
            time.sleep(random.uniform(0.5, 1.5))
        ensure_logged_in(driver)  # check login after break

# -----------------------------
# Selenium setup
# -----------------------------
def init_driver():
    '''Creates a ChromeOptions object to configure the
    Chrome browser. Sets detach to True so that 
    the window stays open. Launches the browser and 
    returns a Selenium driver object that we can control
    the browser'''
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

def manual_login(driver):
    '''Opens the login page in the browser, prints 
    a message asking to log in manually, and pauses execution
    until you press Enter. You should press enter after you sign
    in manually'''
    driver.get("https://www.beeradvocate.com/community/login/")
    print("🔑 Please log in manually.")
    input("Enter after logging in")
    print("✅ Logged in!")

def ensure_logged_in(driver):
    """Check if BeerAdvocate redirected us to login, and pause if so."""
    if "login" in driver.current_url.lower():
        print("🔑 Session expired! Please log in again.")
        input("Press Enter after logging back in...")
        print("✅ Logged in again. Resuming scrape...")

def save_html_cache(beer_name, html):
    '''Finds or creates a chached_pages folder that stores snapshots of
    the beer review page's HTML that Selenium fetched. 
    It saves progress locally in case scrapping is interrupted. Converts
    the beer's name into a safe filename by including underscores,
    builds the file path with that name and a .html exension. '''
    folder = "cached_pages"
    os.makedirs(folder, exist_ok=True)
    safe_name = "".join(c if c.isalnum() else "_" for c in beer_name)
    path = os.path.join(folder, f"{safe_name}.html")
    with open(path, "w", encoding="utf-8") as f:
        f.write(html)
    return path

# -----------------------------
# Scrape reviews safely
# -----------------------------
def scrape_all_reviews(driver, beer_name, beer_url, max_reviews=40):  
    '''
    Initializes an empty list of reviews and a counter of reviews.
    Enters a loop to load pages of reviews for a beer URL.
    Tries up to 3 times to load the page and if it fails it skips
    that beer. If it loads, it opens the beer page with Selenium, 
    checks if we are logged in, waits 40 seconds for at least one 
    review score element. Calls slow_scroll to simulate human scrolling.
    Saves the page's HTML locally and finds all review containers extracting
    the rating and the review text. Appends the clean review and rating to the
    all_reviews list and stops if the list has 40 reviews. 

    Finds "next" and "last" page links stopping if next is also the last page.
    Otherwise it updates the beer_url and waits a short random delay. 

    Return the list of scrapped reviews with their ratings. 
    '''
    all_reviews = []
    reviews_count = 0

    while True:
        retries = 0
        page_loaded = False
        while not page_loaded and retries < 3:
            try:
                driver.get(beer_url)

                # check login
                ensure_logged_in(driver)

                WebDriverWait(driver, 40).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.BAscore_norm'))
                )
                slow_scroll(driver)
                page_loaded = True
            except Exception as e:
                print(f"❌ Failed to load page. Retrying... ({retries+1}/3)")
                retries += 1
                time.sleep(3)
        
        if not page_loaded:
            print(f"⚠️ Failed to load page after 3 attempts. Skipping {beer_name}.")
            break

        # Save page HTML
        save_html_cache(beer_name, driver.page_source)

        review_divs = driver.find_elements(By.CSS_SELECTOR, 'div[id^="rating_fullview_content_"]')
        for div in review_divs:
            try:
                rating = div.find_element(By.CSS_SELECTOR, '.BAscore_norm').text.strip()
                review_text_div = div.find_element(By.CSS_SELECTOR, 'div[style*="line-height:1.4"]')
                full_text = review_text_div.get_attribute('innerHTML')
                clean_review = full_text.split('<br><br>Review date:')[0].split('<br><br>Bottle date:')[0].replace('<br>', '\n').strip()
                if clean_review:
                    all_reviews.append({'review': clean_review, 'rating': rating})
                    reviews_count += 1
                    if reviews_count >= max_reviews:
                        print(f"✅ Reached {max_reviews}-review cap for {beer_name}.")
                        return all_reviews
            except:
                continue

        print(f"➡️ Scraped {len(review_divs)} reviews, total so far: {reviews_count}")

        # Pagination
        try:
            next_link = driver.find_element(By.LINK_TEXT, 'next')
            last_link = driver.find_element(By.LINK_TEXT, 'last')
            next_href = next_link.get_attribute('href')
            last_href = last_link.get_attribute('href')
            if next_href == last_href:
                print(f"✅ Last page of reviews reached for {beer_name}.")
                break
            else:
                beer_url = next_href
                human_delay((3, 7))
        except:
            print(f"✅ No next/last link found. Finished {beer_name}.")
            break

    return all_reviews

# -----------------------------
# Main workflow
# -----------------------------
def main():
    '''
    Initializes browser, sets up output CSV and limits to reviews. 
    If the CSV exists we load it into all_reviews_data. 
    Tracks which beers were scrapped already to skip them.
    Opens loging page and waits for us to login, gets the list of
    250 top beers, and loops through them, stops if the reviews
    reach the maximum. Skips the beers already scrapped, ensures
    we are logged in, calls scrape_all_reviews to get 40 reviews
    for that beer. Adds the beer name to each review record.
    Appends new reviews to all_reviews_data. Saves progress to CSV,
    including the reviews for beer for which a page failed to load
    at some point. Add a random humal delay of 60-180 seconds
    and takes long breaks every 4 beers using long_break_after_beers
    '''
    driver = init_driver()
    csv_path = 'test.csv'
    MAX_TOTAL_REVIEWS = 15000

    # Resume progress if file exists
    if os.path.exists(csv_path):
        df_existing = pd.read_csv(csv_path)
        all_reviews_data = df_existing.to_dict("records")
        scraped_beers = set(df_existing["product_name"].unique())
        print(f"🔄 Resuming: {len(scraped_beers)} beers already scraped, {len(all_reviews_data)} reviews.")
    else:
        all_reviews_data = []
        scraped_beers = set()

    try:
        manual_login(driver)
        beer_list = get_top_beer_urls()

        for i, beer in enumerate(beer_list):
            if len(all_reviews_data) >= MAX_TOTAL_REVIEWS:
                print(f"🛑 Reached total review cap of {MAX_TOTAL_REVIEWS}. Stopping.")
                break

            beer_name = beer['name']
            beer_url = beer['url']

            if beer_name in scraped_beers:
                print(f"⏭️ Skipping {beer_name}, already scraped.")
                continue

            ensure_logged_in(driver)  # check before scraping

            print(f"\n🍺 Scraping beer {i+1}/{len(beer_list)}: {beer_name}")
            revs = scrape_all_reviews(driver, beer_name, beer_url, max_reviews=40)  # now 40 max reviews

            for r in revs:
                r['product_name'] = beer_name
            all_reviews_data.extend(revs)

            df = pd.DataFrame(all_reviews_data)
            df.to_csv(csv_path, index=False)

            print(f"💾 Progress saved. Total reviews so far: {len(all_reviews_data)}")

            # Human delay and long break
            human_delay((60, 180))
            long_break_after_beers(driver, i)

    finally:
        driver.quit()
        print(f"\n💾 Total reviews collected: {len(all_reviews_data)}")


if __name__ == "__main__":
    main()


## Key Implementation Features

### Hybrid Scraping Strategy
- **Static scraping**: Top 250 beers list using requests + BeautifulSoup
- **Dynamic scraping**: Individual reviews using Selenium (handles login + JavaScript)

### Anti-Detection System
- **Human delays**: Random 2-5 second pauses between requests
- **Scroll simulation**: Mimics natural reading behavior with variable scrolling
- **Long breaks**: 3-5 minute breaks every 4 beers to avoid IP blocking

### Fault Tolerance
- **3 retry attempts** for failed page loads
- **Session monitoring**: Detects login expiration and prompts re-authentication
- **Progress persistence**: Resume scraping from interruptions using CSV checkpoints

### Data Pipeline
- **Clean extraction**: Removes HTML artifacts and metadata from reviews
- **Structured output**: product_name, product_review, user_rating columns
- **Quality control**: 40 review limit per beer, validates non-empty content



In [7]:
df = pd.read_csv("scrap_data.csv")
df.rename(columns={'rating': 'Rating', 'review': 'Review_text', 'product_name':'Beer_name'}, inplace=True)
df.head()

,Review_text,Rating,Beer_name
0,Good,4.41,Kentucky Brunch Brand Stout
1,"Pours the purest black color you’ve ever seen,...",4.94,Kentucky Brunch Brand Stout
2,"This beer is intense, and yet, it feels very s...",4.98,Kentucky Brunch Brand Stout
3,2022 vintage poured at fridge temp but tasted ...,4.43,Kentucky Brunch Brand Stout
4,"Sampled at the brewery, this is the 2022 bottl...",4.61,Kentucky Brunch Brand Stout


In [8]:
pd.set_option("display.max_rows", 0)

# Task B: Attribute Discovery & Bag-of-Words Recommendation

We are building the foundation for a bag-of-words recommender by discovering meaningful beer attributes through frequency analysis and lift analysis.

In [9]:
# Drop empty rows
df = df.dropna()

def clean_text(text):
    # Remove non-alphabetic characters, lowercase
    text = re.sub(r"[^a-zA-Z ]", " ", str(text)).lower()
    return text

df['clean_text'] = df['Review_text'].apply(clean_text)

# Domain-specific stop words
domain_stop = set(['beer','bottle','one','like','taste','review','product','drink','head','s','t'])
all_stopwords = STOP_WORDS.union(domain_stop)

def clean_tokens(text):
    tokens = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    return [t for t in tokens if t not in all_stopwords]

df['tokens'] = df['clean_text'].apply(clean_tokens)

all_tokens = [token for tokens in df['tokens'] for token in tokens]
word_freq = Counter(all_tokens)

freq_df = pd.DataFrame(word_freq.items(), columns=['word','frequency']).sort_values(by='frequency', ascending=False).reset_index(drop=True)
##freq_df[:200]

In [10]:
# List of generic words to exclude for attribute analysis
exclude_words = set([
    'good','nice','overall','look','feel','m','ve','o','f','best','one','way','try',
    'don','years','lot','lots','maybe','think','comes','got','doesn','house', 'great',
    'little', 'slightly', 'big' , 'time', 'end','finger','oz'
])

# Take top frequent words, remove generic words
top_candidates = [w for w in freq_df['word'].head(100).tolist() if w not in exclude_words]

##print("Candidate words for lift analysis:\n", top_candidates)

In [11]:
# 1. Candidate attributes 

attributes = [
    # Flavor
    'chocolate','coffee','vanilla','caramel','citrus','grapefruit',
    'tropical','maple','coconut','roasted','sour','tart','fruit','peach',
    'mango','orange','pine','lemon','malts','hops','hop','bitter','bitterness',
    'sweet','sweetness','dry','bourbon','cinnamon','funk','stout','ipa',

    # Mouthfeel / texture
    'smooth','creamy','thick','thin','carbonation','body','mouthfeel',
    'soft','pours','poured','juicy','touch','retention','medium','light',

    # Strength / intensity
    'strong','alcohol','abv','moderate','bodied','bit','hint',

    # Balance / complexity
    'balanced','complex','rich','finish','aroma','nose','flavor','flavors','palate','balance',

    # Appearance
    'dark','black','brown','hazy','color','lacing','white','tan','glass'
]


# 2. Build co-occurrence counts

co_occur = defaultdict(int)
attr_counts = defaultdict(int)

for text in df["clean_text"]:   
    tokens = set(text.split())  # unique words in review
    # Count each attribute if it appears
    for attr in attributes:
        if attr in tokens:
            attr_counts[attr] += 1
    # Count co-occurrence of attribute pairs
    for a, b in combinations(attributes, 2):
        if a in tokens and b in tokens:
            co_occur[(a, b)] += 1


# 3. Convert to dataframe with Lift

results = []
N = len(df)  # total reviews

for (a, b), co_count in co_occur.items():
    if co_count == 0:
        continue
    p_a = attr_counts[a] / N
    p_b = attr_counts[b] / N
    p_ab = co_count / N
    lift = p_ab / (p_a * p_b)
    results.append((a, b, co_count, lift))

lift_df = pd.DataFrame(results, columns=["attr1", "attr2", "co_count", "lift"])

# 4. Filter and sort

lift_df = lift_df[lift_df["lift"] > 1].sort_values(by="lift", ascending=False)

#filter out very low co-occurences
min_frac = 0.001  # 0.1%
min_count = int(min_frac * len(df))  

filtered = lift_df[lift_df["co_count"] >= min_count]
filtered = filtered.sort_values(by="lift", ascending=False)

filtered[:25]

,attr1,attr2,co_count,lift
844,roasted,malts,258,6.220044
1490,tart,funk,300,6.153267
1508,sour,funk,247,5.968833
1493,sour,tart,240,5.459913
2127,peach,mango,242,5.124589
2391,lemon,funk,206,5.067530
1710,tropical,mango,364,4.938356
1970,grapefruit,mango,344,4.726788
1682,grapefruit,pine,263,4.612868
...,...,...,...,...


### Statistical Feature Selection
- **Lift metric**: Measures how much more likely two words co-occur vs. random chance
- **Co-occurrence counting**: Identifies attribute pairs that frequently appear together in reviews
- **Threshold-based filtering**: Removes low-frequency and generic terms to focus on descriptive attributes

### Why Build a Flavor Map?
Flavor descriptors in reviews often use different words to describe the same underlying taste (e.g., *sour* vs. *tart*, *hop* vs. *hops*). Without cleaning, these synonyms inflate redundancy and fragment associations across multiple labels.  
By mapping related terms to canonical names (our **flavor map**), we:
- Reduce noise and ensure consistency in analysis.  
- Make co-occurrence patterns more interpretable.  
- Prevent overcounting pairs that are essentially identical in meaning.  

---

In [12]:
# we see redundancy in attributes as sour/tart as essentially synonyms, in order to avoid redundancy, 
## we map similar flavors to canonical names
flavor_map = {
    "sour": "sour",
    "tart": "sour",         
    "funk": "funk",
    "juicy": "juicy",
    "pine": "pine",
    "grapefruit": "citrus",
    "orange": "citrus",
    "peach": "peach",
    "mango": "mango",
    "coffee": "coffee",
    "vanilla": "vanilla",
    "chocolate": "chocolate",
    "caramel": "caramel",
    "roasted": "roasted",
    "hop": "hop",
    "hops": "hop",
    "bitter": "bitter",
    "bitterness": "bitter",
    "stout": "stout",
    "ipa": "ipa",
    "tropical": "tropical"
}

# Map to canonical flavors
lift_df['attr1_clean'] = lift_df['attr1'].map(flavor_map).fillna(lift_df['attr1'])
lift_df['attr2_clean'] = lift_df['attr2'].map(flavor_map).fillna(lift_df['attr2'])

# Remove rows where both cleaned attrs are the same (optional)
lift_df = lift_df[lift_df['attr1_clean'] != lift_df['attr2_clean']]

# Sort the pair so 'funk-sour' and 'sour-funk' are treated the same
lift_df['pair_sorted'] = lift_df.apply(lambda x: tuple(sorted([x['attr1_clean'], x['attr2_clean']])), axis=1)

# Aggregate on the sorted pairs
agg_df = lift_df.groupby('pair_sorted').agg(
    co_count=('co_count','sum'),
    lift=('lift','mean')  # or recalc if needed
).reset_index()

# Split pair tuple back to two columns for readability
agg_df[['attr1_clean','attr2_clean']] = pd.DataFrame(agg_df['pair_sorted'].tolist(), index=agg_df.index)
agg_df = agg_df.drop(columns='pair_sorted')

# Sort by lift
agg_df = agg_df.sort_values(by='lift', ascending=False)

agg_df[:25]

,co_count,lift,attr1_clean,attr2_clean
1576,258,6.220044,malts,roasted
1306,547,6.061050,funk,sour
1593,242,5.124589,mango,peach
1294,206,5.067530,funk,lemon
1605,364,4.938356,mango,tropical
1463,262,4.589456,juicy,mango
848,1306,4.089921,citrus,mango
1448,196,4.087754,ipa,pine
1469,175,4.073857,juicy,peach
...,...,...,...,...




### Candidate Trios of Attributes

1. **Mango / Tropical / Juicy**  
   - mango → tropical = 4.938  
   - juicy → mango = 4.589  
   - juicy → tropical = 4.012  

   This set shows consistently strong lifts, all above 4. However, "mango" and "tropical" overlap conceptually, which limits interpretive value since they represent similar fruity descriptors.

---

2. **Funk / Sour / Lemon**  
   - funk → sour = 6.061  
   - funk → lemon = 5.068  
   - lemon → sour = 3.816  

   This trio captures three distinct dimensions — fermented funk, sharp sourness, and citrus lemon. Although the lemon --> sour link is slightly weaker, the combination offers a more interpretable and complementary flavor profile.

---

✅ **Selected Trio:** `['Funk', 'Sour', 'Lemon']`


## Bag-of-Words Recommender

In [13]:
# Ensure nltk vader is available
nltk.download('vader_lexicon')
nltk.download('wordnet')
sia = SentimentIntensityAnalyzer()

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/moharchaudhuri/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/moharchaudhuri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


- **Filter (keep) beers with ≥5 reviews**  
  *Why:* Avoid unreliable results from beers with too few reviews.  


In [14]:
## Calculate no. of reviews
beer_review_counts = df.groupby("Beer_name").size()

# 2. Filter beers with enough reviews
eligible_beers = beer_review_counts[beer_review_counts >= 5].index
df_filtered = df[df["Beer_name"].isin(eligible_beers)].reset_index(drop=True)

- **Preprocess reviews (lemmatization)**  
  *Why:* Reduce words to their base forms so variants (e.g., *lemons* → *lemon*) are treated consistently.  


In [15]:
# 1. Preprocess reviews (lemmatize)

def lemmatize_text(text):
    tokens = word_tokenize(str(text).lower())
    return " ".join([lemmatizer.lemmatize(t) for t in tokens])

df_filtered["clean_text_lem"] = df_filtered["clean_text"].fillna("").apply(lemmatize_text)

- **Expand attributes with variants (e.g., sour → tart, tartness)**  
  *Why:* Capture different user wordings for the same attribute to improve matching.  


In [16]:
# 2. Expand user attributes with common variants

user_attributes = ["sour", "funk", "lemon"]
attribute_variants = {
    "sour": ["sour", "sours", "sourish", "tart", "tartness"],
    "funk": ["funk", "funky", "funkiness"],
    "lemon": ["lemon", "lemons", "lemony"]
}

# Flatten and lemmatize
expanded_attrs = [v for attr in user_attributes for v in attribute_variants[attr]]
expanded_attrs_lem = [lemmatizer.lemmatize(attr.lower()) for attr in expanded_attrs]

# Build query string
query = " ".join(expanded_attrs_lem)

- **Vectorize reviews with Bag-of-Words**  
  *Why:* Convert text into numerical features for similarity calculations.  


In [17]:
# Bag-of-Words (unigrams). Increase max_features if many zeros.
vectorizer = CountVectorizer(stop_words="english") 
X_reviews = vectorizer.fit_transform(df_filtered["clean_text_lem"])    

- **Aggregate to beer-level representation**  
  *Why:* Summarize all reviews for each beer into one vector for comparison.  


In [18]:
# Aggregate to beer-level
beer_idx = df_filtered.groupby("Beer_name").indices  # dict: beer -> list of review indices
beer_names, rows = [], []
for beer, idxs in beer_idx.items():
    summed = X_reviews[list(idxs)].sum(axis=0)
    rows.append(np.asarray(summed).ravel())
    beer_names.append(beer)
beer_matrix = np.vstack(rows)

- **Compute sentiment per review, then average per beer**  
  *Why:* Capture overall tone (positive/negative) toward each beer, beyond just attribute mentions.  


In [19]:
# 4. Compute per-review sentiment
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['sentiment'] = df_filtered['Review_text'].fillna('').apply(lambda t: sia.polarity_scores(t)['compound'])

# Average sentiment per beer
beer_sentiment = np.array([df_filtered.loc[idxs, 'sentiment'].mean() for idxs in beer_idx.values()])

In [20]:
# Filer for review_counts
beer_review_counts = df_filtered.groupby("Beer_name").size().reindex(eligible_beers).values


- **Build query vector from attributes & compute cosine similarity**  
  *Why:* Identify beers whose reviews are textually similar to the user’s desired attributes.  


In [21]:
# 5. Build query vector and compute cosine similarity
query_vec = vectorizer.transform([query])
cos_sim = cosine_similarity(query_vec, beer_matrix).flatten()  

- **Combine similarity and sentiment into final score**  
  *Why:* Ensure recommendations balance relevance (similarity) with positivity (sentiment).  


In [22]:
def compute_final_score(cos_sim, beer_sentiment, alpha=0.8):
    """
    Compute final score for beers based on flavor similarity, sentiment, and optional rating.
    
    Parameters:
    - cos_sim: array of raw cosine similarity per beer
    - beer_sentiment: array of raw sentiment per beer [-1,1]
    - alpha: weight for flavor match (0-1)
    
    Returns:
    - final_score: array of final scores
    """
    
    # Normalize sentiment to [0,1]
    sentiment_norm = (beer_sentiment + 1) / 2
    
    # Normalize sentiment to [0,1]
    sentiment_norm = (beer_sentiment + 1) / 2
    
    # Blend flavor similarity with normalized sentiment
    final_score = alpha * cos_sim + (1 - alpha) * sentiment_norm
    
    return final_score

In [23]:
final_score = compute_final_score(cos_sim, beer_sentiment)


beer_docs = pd.DataFrame({
    "Beer_name": beer_names,
    "similarity_score": cos_sim,
    "sentiment": beer_sentiment,
    "review_count": beer_review_counts,
    "score": final_score
})

ranked = beer_docs.sort_values("score", ascending=False).reset_index(drop=True)
top20 = ranked.head(20).copy()
top20["Recommendation"] = ["✅ Top 1", "✅ Top 2", "✅ Top 3"] + ["Contender"] * 17

from IPython.display import display
display(top20[["Beer_name", "review_count", "sentiment", "similarity_score", "score", "Recommendation"]])


,Beer_name,review_count,sentiment,similarity_score,score,Recommendation
0,Cellarman Barrel Aged Saison,40,0.695227,0.378010,0.471931,✅ Top 1
1,Saison Du Fermier,55,0.746393,0.367264,0.468451,✅ Top 2
2,R&D Sour Fruit (Very Sour Blackberry),34,0.792650,0.332710,0.445433,✅ Top 3
3,Aurelian Lure,14,0.580629,0.352425,0.440002,Contender
4,Cable Car,22,0.708941,0.325971,0.431671,Contender
5,Saison Bernice,37,0.709230,0.322511,0.428932,Contender
6,Duck Duck Gooze,24,0.628367,0.304639,0.406548,Contender
7,The Broken Truck,60,0.634823,0.301369,0.404577,Contender
8,Oude Geuze Golden Blend,40,0.723158,0.289106,0.403601,Contender
...,...,...,...,...,...,...


### Recommendation Results

- **Top 3 Beers**  
  The top-ranked beers balance strong similarity to the target flavor attributes (*funk, sour, lemon*) with consistently positive sentiment.  
  - **Cellarman Barrel Aged Saison** (Top 1) – Strong overall score driven by good similarity and balanced sentiment.  
  - **Saison Du Fermier** (Top 2) – Very positive sentiment (0.75) helps push this beer into the top tier.  
  - **R&D Sour Fruit (Very Sour Blackberry)** (Top 3) – Strong sentiment (0.79), though slightly lower similarity, still secures a top recommendation.  

- **Contenders**  
  Beers such as *Aurelian Lure*, *Cable Car*, and *Saison Bernice* show moderate similarity and sentiment but fall below the top 3 in the combined score.  
  These are still promising candidates for exploration but less directly aligned than the leaders.  

- **Notable Observations**  
  - Beers like *Clover* and *Oude Geuze Cuvée Armand & Gaston* have very high sentiment (>0.78) but weaker similarity to the query, which prevents them from reaching the top tier.  
  - Conversely, *Aurelian Lure* shows solid similarity (0.35) but more mixed sentiment (0.58), limiting its score.  
  - This highlights the **trade-off between textual similarity and reviewer positivity**—both matter for final ranking.  

**Interpretation:** The model is effectively surfacing saisons and sour-style beers, especially those with descriptors aligned to *funk / sour / lemon*. The top 3 are strong recommendations, while the contenders serve as a broader shortlist.  


##  Task C: Word Embeddings Results Analysis

The word embeddings approach with spaCy produces beer recommendations that are **semantically different** from the bag-of-words method (Task B), using our selected attributes: **Sour, Funk, Lemon**.


In [24]:
# Initialize tokenizer
# Loading medium model, stated in the instructions
nlp = spacy.load("en_core_web_md")
spacy_tokenizer = nlp.tokenizer

In [25]:
#function to get word embeddings 
def prep(x):
     z=spacy_tokenizer(x) 
     z=nlp(z).vector.reshape(300,) 
#the .vector vectorizes into a word embedding and then reshapes back to 300 dim

- **Define embedding function (`get_embedding`)**  
  *Why:* Converts any text (review or query) into a fixed-size 300-dimensional vector, enabling comparison between beers and user attributes.  


In [26]:
def get_embedding(text: str) -> np.ndarray:
    """Return the 300-d spaCy vector for a string."""
    return nlp(text).vector

- `get_embedding()` returns a 300-dimensional embedding for a string.

- For multi-word text, SpaCy averages the embeddings of all tokens to produce a single vector.

In [27]:
# takes around 3 mins to run - go grab a beer!
df_filtered["spacy_embedding"] = df_filtered["clean_text"].apply(get_embedding) # goes row by row applying get_embedding

- **Compute embeddings for all reviews**  
  Apply the embedding function row by row to the cleaned review text.  
  Transforms unstructured text into numerical vectors that can be averaged and compared.  


In [28]:
# Grouping By beer
beer_emb = (
    df_filtered.groupby("Beer_name")
       .agg(
           embedding=("spacy_embedding", lambda vs: np.mean(np.vstack(vs), axis=0)),
           n_reviews=("Review_text", "size"),
           avg_sentiment=("sentiment", "mean"),   # [-1, 1] averaged across reviews
       )
       .reset_index())

beer_emb[:10]

,Beer_name,embedding,n_reviews,avg_sentiment
0,10 Year Barleywine,"[-0.22058198, -0.026772585, -1.8499376, 0.2039...",7,0.699329
1,4th Anniversary,"[-1.0424777, 0.41012728, -2.074242, 0.44822305...",42,0.694293
2,A Deal With The Devil - Double Oak-Aged,"[-1.2144722, 0.4047579, -2.0531967, 0.53321654...",48,0.768896
3,A Deal With The Devil - Triple Oak-Aged,"[-1.0170851, 0.36681813, -2.133802, 0.3024488,...",44,0.716530
4,Aaron,"[-1.212208, 0.33313152, -1.7961754, 0.7020584,...",28,0.668432
5,Abner,"[-1.212755, 0.521003, -2.1834424, 0.30859357, ...",60,0.701982
6,Abrasive Ale,"[-1.2223977, 0.60559547, -2.0106401, 0.5724674...",40,0.690035
7,Abraxas,"[-1.0470912, 0.49509925, -2.2511003, 0.5408371...",40,0.772645
8,Abraxas - Barrel-Aged,"[-1.5121418, 0.28372142, -2.0436356, 0.4792746...",60,0.654937
9,Abraxas - Coffee,"[-1.1454557, 0.39163977, -2.0457091, 0.5848397...",40,0.691290


- Group all reviews for each beer.

- **Average the embeddings** of all reviews --> one beer-level vector.

- Also calculate:

    - `avg_rating` --> average star rating

    - `n_reviews` --> number of reviews

- Creates a single semantic “profile vector” per beer, summarizing how it is described across reviews.  


In [29]:
beer_matrix = np.vstack(beer_emb["embedding"].values)

query_vec = get_embedding(query) # One attribute vector
query_vec_2d = query_vec.reshape(1, -1)

# Compute cosine similarity between query and all beers
spacy_sims = cosine_similarity(query_vec_2d, beer_matrix).flatten()



- **Build query vector for user attributes (`funk sour lemon`)**  
  *Why:* Embeds the target flavor profile into the same semantic space as the beers, allowing direct comparison.  

- **Compute cosine similarity between query and beer embeddings**  
  *Why:* Measures how close each beer’s description is to the desired attributes in semantic space. Higher values = more similar.  

- **Combine similarity with sentiment into final score**  
  *Why:* Prevents recommending beers that match the flavor profile but are poorly received, and balances relevance with positivity.  


In [30]:
final_score = compute_final_score(
    cos_sim=spacy_sims,
    beer_sentiment=beer_emb['avg_sentiment'],  
    alpha=0.8
)


beer_docs = pd.DataFrame({
    "Beer_name": list(beer_idx.keys()),
    "similarity_score": spacy_sims,   
    "sentiment": beer_emb['avg_sentiment'],   
    "score": final_score
})

ranked = beer_docs.sort_values("score", ascending=False).reset_index(drop=True)
top20 = ranked.head(20).copy()
top20["Recommendation"] = ["✅ Top 1", "✅ Top 2", "✅ Top 3"] + ["Contender"] * 17

from IPython.display import display
display(top20[["Beer_name", "sentiment", "similarity_score", "score", "Recommendation"]])

,Beer_name,sentiment,similarity_score,score,Recommendation
0,Peche Du Fermier,0.712250,0.485078,0.559288,✅ Top 1
1,Hommage,0.670662,0.489641,0.558779,✅ Top 2
2,Supplication,0.683192,0.486203,0.557281,✅ Top 3
3,Double Dry Hopped Mylar Bags,0.692670,0.483670,0.556203,Contender
4,Speedway Stout - Vietnamese Coffee - Rye Whisk...,0.839259,0.464771,0.555743,Contender
5,Moment Of Clarity,0.751922,0.475601,0.555673,Contender
6,Beyond Good And Evil,0.723050,0.478898,0.555423,Contender
7,Cellarman Barrel Aged Saison,0.695227,0.481477,0.554704,Contender
8,Nectarine Premiere,0.682632,0.482387,0.554173,Contender
...,...,...,...,...,...


### Recommendation Results (spaCy Embeddings)

- **Top 3 Beers**  
  - **Peche Du Fermier (Top 1)** – Strong similarity (0.485) and solid sentiment (0.71) yield the highest overall score.  
  - **Hommage (Top 2)** – Slightly higher similarity (0.490) than Top 1, but lower sentiment (0.67) balances it to second place.  
  - **Supplication (Top 3)** – Consistent sentiment (0.68) and similarity (0.486) secure a stable Top 3 placement.  

- **Contenders**  
  - *Double Dry Hopped Mylar Bags* – Nearly identical similarity to the top beers (0.484) with sentiment (0.69), just shy of breaking into Top 3.  
  - *Speedway Stout – Vietnamese Coffee – Rye Whiskey* – Extremely high sentiment (0.84, the highest in the table) but slightly lower similarity (0.465), showing it’s loved by drinkers but less directly tied to the *funk / sour / lemon* profile.  
  - Other strong contenders (*Moment Of Clarity*, *Beyond Good And Evil*, *Cellarman Barrel Aged Saison*) cluster closely in both sentiment and similarity, indicating a competitive mid-tier.  

- **Notable Observations**  
  - Scores are **very tightly packed** among the top 15, suggesting the embedding method surfaces many semantically relevant beers with only marginal differences.  
  - High-sentiment beers (e.g., *All That Is And All That Ever Will Be*, *Medianoche – Coconut*) rank lower on similarity, confirming the importance of balancing flavor relevance with positivity.  
  - Compared to the Bag-of-Words method, these results are **less dominated by explicit “sour/funk/lemon” mentions** and instead capture semantic neighbors, bringing in more diverse but still contextually relevant beers.  

**Interpretation:** The embedding-based model produces a **dense cluster of top candidates** with minimal score gaps. While the Top 3 are solid recommendations, several contenders (notably *Double Dry Hopped Mylar Bags* and *Speedway Stout*) could easily compete depending on whether **flavor alignment** or **sentiment strength** is prioritized.  


## Top 5 Comparison: Bag-of-Words vs. spaCy Embeddings

| Rank | Bag-of-Words (BoW)                          | Sentiment | Similarity | Score   | spaCy Embeddings                        | Sentiment | Similarity | Score   |
|------|---------------------------------------------|-----------|------------|---------|-----------------------------------------|-----------|------------|---------|
| 1    | Cellarman Barrel Aged Saison                | 0.695     | 0.378      | 0.472   | Peche Du Fermier                        | 0.712     | 0.485      | 0.559   |
| 2    | Saison Du Fermier                           | 0.746     | 0.367      | 0.468   | Hommage                                 | 0.671     | 0.490      | 0.559   |
| 3    | R&D Sour Fruit (Very Sour Blackberry)       | 0.793     | 0.333      | 0.445   | Supplication                            | 0.683     | 0.486      | 0.557   |
| 4    | Aurelian Lure                               | 0.581     | 0.352      | 0.440   | Double Dry Hopped Mylar Bags            | 0.693     | 0.484      | 0.556   |
| 5    | Cable Car                                   | 0.709     | 0.326      | 0.432   | Speedway Stout – Vietnamese Coffee – Rye Whiskey | 0.839     | 0.465      | 0.556   |

---

### Key Insights
- **BoW method** emphasizes beers with explicit mentions of *sour/funk/lemon*, surfacing saisons and sour fruit beers prominently.  
- **Embedding method** surfaces semantically related beers, including stouts and blends, that don’t necessarily use the exact keywords but share contextual meaning.  
- **Similarity trade-off:** BoW favors **direct textual overlap**, while embeddings reward **contextual closeness** even if exact descriptors differ.  

**Interpretation:**  
- **BoW = precise keyword matching** (narrow but exact).  
- **Embeddings = semantic generalization** (broader but contextually relevant).  
- Combining both methods provides complementary perspectives: BoW anchors on explicit flavor tags, embeddings broaden the net to capture beers described with richer or alternative language.  


# Custom Embeddings for Beer Recommendations

Here in order to generate recommendations tailored to beer drinkers, we trained **custom word embeddings** directly on our review data rather than relying on generic models like `spaCy`. By doing so, we capture the unique vocabulary of the craft beer world — words like `funky`, `saison`, or `tart` that matter to enthusiasts but don’t carry the same weight in general English. We then layered in **TF-IDF** weighting, which is key to making the model smarter: it downplays common filler words such as beer or drink and amplifies distinctive flavor terms like sour or lemony. Each review is transformed into a weighted average of its word vectors, producing richer signals about what truly defines that beer. At the beer level, these vectors are aggregated to create a semantic “flavor profile,” and we embed the user’s query (e.g., **funk, sour, lemon**) in the same space. Cosine similarity shows how closely each beer matches the desired profile, and we combine that with sentiment so that recommendations not only taste right on paper but are also loved by drinkers. *Compared to off-the-shelf embeddings, this approach gives sharper, more relevant recommendations because it is trained on the language our reviewers actually use and weights the words that matter most.*

## Key Intuition
- **Generic embeddings (spaCy)** → capture broad language patterns, but may miss niche beer vocabulary.  
- **Custom embeddings (Word2Vec + TF-IDF)** → trained on *our own reviews*, tuned to emphasize flavor words.
- We apply **TF-IDF weighting** to guide the embeddings:  
  - 🔹 **Downweights common filler words** (*beer*, *drink*) that add noise.  
  - 🔹 **Amplifies distinctive flavor terms** (*sour*, *lemony*, *funky*) that actually define taste.  
  - ⚡ Ensures the model emphasizes **what makes a beer unique**.  

- Each review is converted into a **TF-IDF–weighted embedding**, then aggregated into a **beer-level flavor profile**.  

- The user’s query (e.g., *funk, sour, lemon*) is embedded in the same space, and we use **cosine similarity + sentiment** to rank beers.  

- **Effect on recommendations:** Top 3 shifts because the model now highlights beers that reviewers describe in *domain-specific ways*, even if those words don’t appear in generic embeddings.  


In [32]:
# -------------------------
# Step 0: Setup
# -------------------------
vec_size = 300  # Word2Vec vector size

# Train Word2Vec on tokenized reviews
sentences_tokenized = df_filtered['tokens'].tolist()  # preprocessed token lists
model_word2vec = Word2Vec(
    sentences_tokenized,
    vector_size=vec_size,
    window=5,
    min_count=3,
    sg=1
)
wv = model_word2vec.wv

# -------------------------
# Step 1: Compute TF-IDF on token strings
# -------------------------
df_filtered['tokens_str'] = df_filtered['tokens'].apply(lambda x: " ".join(x))
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_filtered['tokens_str'])
tfidf_vocab = tfidf.get_feature_names_out()
idf_dict = dict(zip(tfidf_vocab, tfidf.idf_))

# -------------------------
# Step 2: Helper: weighted Word2Vec embedding for a review
# -------------------------
def weighted_w2v_embedding(tokens):
    vecs, weights = [], []
    for t in tokens:
        if t in wv.key_to_index and t in idf_dict:
            vecs.append(wv[t])
            weights.append(idf_dict[t])
    if vecs:
        vecs = np.stack(vecs)
        weights = np.array(weights)[:, None]
        return np.sum(vecs * weights, axis=0) / np.sum(weights)
    else:
        return np.zeros(vec_size)

# -------------------------
# Step 3: Compute embeddings for all reviews
# -------------------------
df_filtered['custom_embeddings'] = df_filtered['tokens'].apply(weighted_w2v_embedding)

# -------------------------
# Step 4: Aggregate to beer-level embeddings
# -------------------------
beer_groups = df_filtered.groupby("Beer_name")["custom_embeddings"].apply(
    lambda s: np.mean(np.stack(s.values), axis=0)
)
beer_names = beer_groups.index.tolist()
beer_matrix = np.vstack(beer_groups.values)

# -------------------------
# Step 5: Compute query embedding (TF-IDF weighted)
# -------------------------
query_tokens = expanded_attrs_lem  # your lemmatized attribute list
query_vecs, weights = [], []
for t in query_tokens:
    if t in wv.key_to_index and t in idf_dict:
        query_vecs.append(wv[t])
        weights.append(idf_dict[t])

if query_vecs:  # avoid errors if no overlap
    query_vec = np.sum(np.stack(query_vecs) * np.array(weights)[:, None], axis=0) / np.sum(weights)
else:
    query_vec = np.zeros(vec_size)
query_vec = query_vec.reshape(1, -1)

# -------------------------
# Step 6: Compute cosine similarity & final score
# -------------------------
beer_sentiment = df_filtered.groupby("Beer_name")['sentiment'].mean().reindex(beer_names).values
beer_review_counts = df_filtered.groupby("Beer_name").size().reindex(beer_names).values

custom_sims = cosine_similarity(query_vec, beer_matrix).flatten()

final_score = compute_final_score(
    cos_sim=custom_sims,
    beer_sentiment=beer_sentiment,
    alpha=0.8
)

# -------------------------
# Step 7: Build result DataFrame
# -------------------------
beer_docs = pd.DataFrame({
    "Beer_name": beer_names,
    "num_reviews": beer_review_counts,
    "similarity_score": custom_sims,
    "sentiment": beer_sentiment,
    "score": final_score
}).sort_values("score", ascending=False).reset_index(drop=True)

beer_docs["Recommendation"] = ["✅ Top 1", "✅ Top 2", "✅ Top 3"] + ["Contender"] * (len(beer_docs)-3)

from IPython.display import display
display(beer_docs.head(20))


,Beer_name,num_reviews,similarity_score,sentiment,score,Recommendation
0,West Ashley,26,0.851142,0.781604,0.859074,✅ Top 1
1,Cellarman Barrel Aged Saison,40,0.859849,0.695227,0.857402,✅ Top 2
2,Saison Du Fermier,55,0.852119,0.746393,0.856334,✅ Top 3
3,Abricot Du Fermier,57,0.848084,0.769482,0.855415,Contender
4,Peche Du Fermier,26,0.853755,0.712250,0.854229,Contender
5,Thicket,14,0.843544,0.754886,0.850324,Contender
6,Framboos,60,0.839426,0.733045,0.844846,Contender
7,Framboise Du Fermier,58,0.841619,0.714476,0.844743,Contender
8,Montmorency Vs Balaton,37,0.830365,0.769230,0.841215,Contender
...,...,...,...,...,...,...


### Recommendation Results (Custom Word2Vec + TF-IDF)

- **Top 3 Beers**  
  - **West Ashley (Top 1)** – Very strong similarity (0.85) combined with high sentiment (0.78) makes this the top recommendation.  
  - **Cellarman Barrel Aged Saison (Top 2)** – Slightly higher similarity (0.86) but lower sentiment (0.70) than Top 1, balancing out at second place.  
  - **Saison Du Fermier (Top 3)** – Strong sentiment (0.75) plus excellent similarity (0.85) secures a stable Top 3 position.  

- **Contenders**  
  - *Abricot Du Fermier* and *Peche Du Fermier* — very close scores to the Top 3, reflecting high similarity and strong sentiment; either could break into the top tier depending on weighting.  
  - *Thicket* and *Framboos* — good balance of sentiment (0.75 / 0.73) and similarity (~0.84), showing they align well with the funk/sour/lemon profile.  
  - *Montmorency vs Balaton* — slightly lower similarity (0.83) but very high sentiment (0.77), keeping it competitive.  

- **Notable Observations**  
  - Scores are **very tightly clustered** at the top (0.835–0.859), meaning many beers are strong matches; the top rankings could shift with small changes in weighting.  
  - Custom embeddings appear to highlight **fruit-forward saisons and fruited sours**, which matches the targeted query (*funk, sour, lemon*).  
  - Compared to spaCy embeddings, this method surfaces more niche saisons (e.g., *Abricot Du Fermier*, *Peche Du Fermier*) that may not be emphasized in generic embeddings.  

**Interpretation:** The custom Word2Vec + TF-IDF approach produces a Top 3 dominated by **high-similarity, sour-forward saisons**, with several close contenders that could easily rotate in. The use of **TF-IDF ensures that flavor-defining words drive the recommendations**, making the results sharper and more domain-relevant than those from generic embeddings.  



# Task E: Comparing Rating-Only vs. Attribute-Based Recommendations


### 1. Top 3 by Rating Only
- **10 Year Barleywine** (avg rating: 4.98, sentiment: 0.70, similarity: 0.59)  
- **O.W.K.** (avg rating: 4.92, sentiment: 0.69, similarity: 0.64)  
- **M.J.K.** (avg rating: 4.84, sentiment: 0.52, similarity: 0.64)  

➡️ These beers are highly rated overall but show **moderate similarity** (0.59–0.64) to the target attributes (*funk, sour, lemon*). In particular, **M.J.K.** has weaker sentiment (0.52), suggesting mixed or polarized reception despite the high rating.  

---

### 2. Top 3 by Attribute-Based Model (TF-IDF Weighted Word2Vec)
- **West Ashley** (similarity: 0.85, sentiment: 0.78, avg rating: 4.53)  
- **Cellarman Barrel Aged Saison** (similarity: 0.86, sentiment: 0.70, avg rating: 4.45)  
- **Saison Du Fermier** (similarity: 0.85, sentiment: 0.75, avg rating: 4.51)  

➡️ These beers combine **very high similarity scores** (>0.85) with **positive sentiment** (>0.70), aligning closely with the user’s flavor request. Ratings are slightly lower than the top-3-by-rating beers but still strong (4.4–4.5).  

---

### 3. Would Rating-Only Recommendations Meet the User’s Needs?
- **No, not fully.**  
  - Rating-only recommendations surface excellent beers in absolute terms, but **they are not necessarily aligned with the requested flavor profile** (*funk, sour, lemon*).  
  - For example, *10 Year Barleywine* and *O.W.K.* are top-rated, but barleywines tend to emphasize sweetness, caramel, or alcohol warmth — not sour/funky/lemony notes.  
  - This creates a mismatch: **high quality beers, but not relevant to the user’s stated taste.**

---
- **Rating-Only Approach:** Optimizes for global quality perception but ignores user context --> risk of irrelevant recommendations.  
- **Attribute-Based Approach:** Prioritizes beers that explicitly match desired flavor attributes while also considering sentiment → ensures relevance and enjoyment.  
- **Key Trade-off:** Attribute-based results may rank slightly lower in global ratings, but they **better meet the user’s needs** because they filter for the specific profile requested.  

---

✅ **Conclusion:**  
If we only choose the 3 highest-rated beers, the recommendations would not fully satisfy the user seeking *funky, sour, lemony* beers. Attribute-driven models like TF-IDF weighted Word2Vec are essential because they **balance quality (ratings/sentiment) with relevance (flavor similarity)**, ensuring recommendations match both taste preferences and positive reception.  

In [33]:
# -----------------------------
# Step 0: Ensure 'Rating' is numeric and create avg_rating map
# -----------------------------
df_filtered['Rating'] = pd.to_numeric(df_filtered['Rating'], errors='coerce')
rating_map = df_filtered.groupby("Beer_name")["Rating"].mean()

# -----------------------------
# Step 1: Create maps from your TF-IDF weighted Word2Vec pipeline
# -----------------------------
similarity_map = beer_docs.set_index('Beer_name')['similarity_score']
sentiment_map = beer_docs.set_index('Beer_name')['sentiment']
final_score_map = beer_docs.set_index('Beer_name')['score']

# -----------------------------
# Step 2: Build 'Top Rated' DataFrame (rating-first view)
# -----------------------------
top_rated_df = pd.DataFrame({
    'Beer_name': rating_map.index,
    'avg_rating': rating_map.values,
    'similarity_score': [similarity_map.get(b, 0) for b in rating_map.index],
    'sentiment': [sentiment_map.get(b, 0) for b in rating_map.index],
    'score': [final_score_map.get(b, 0) for b in rating_map.index]
})

top_rated_df = top_rated_df.sort_values("avg_rating", ascending=False)
final_cols = ['Beer_name', 'avg_rating', 'similarity_score', 'sentiment', 'score']

# -----------------------------
# Step 3: Build 'Attribute-Based' DataFrame (recommender-first view)
# -----------------------------
ranked_df = beer_docs.copy()
ranked_df['avg_rating'] = ranked_df['Beer_name'].map(rating_map).fillna(0)
ranked_df = ranked_df[final_cols].sort_values("score", ascending=False)

# -----------------------------
# Step 4: Add recommendations for display (optional)
# -----------------------------
ranked_df["Recommendation"] = ["✅ Top 1", "✅ Top 2", "✅ Top 3"] + ["Contender"] * (len(ranked_df)-3)

# -----------------------------
# Step 5: Display final comparisons
# -----------------------------
print("✅ === Top 3 by RATING ONLY (with all metrics) ===")
display(top_rated_df[final_cols].head(3))

print("\n✅ === Top 3 by ATTRIBUTES (TF-IDF weighted Word2Vec) ===")
display(ranked_df.head(3))

✅ === Top 3 by RATING ONLY (with all metrics) ===


,Beer_name,avg_rating,similarity_score,sentiment,score
0,10 Year Barleywine,4.980000,0.593064,0.699329,0.644384
169,O.W.K.,4.920000,0.638977,0.692100,0.680392
151,M.J.K.,4.840588,0.636552,0.520876,0.661329



✅ === Top 3 by ATTRIBUTES (TF-IDF weighted Word2Vec) ===


,Beer_name,avg_rating,similarity_score,sentiment,score,Recommendation
0,West Ashley,4.531538,0.851142,0.781604,0.859074,✅ Top 1
1,Cellarman Barrel Aged Saison,4.452250,0.859849,0.695227,0.857402,✅ Top 2
2,Saison Du Fermier,4.509636,0.852119,0.746393,0.856334,✅ Top 3


## Comparison of Recommendation Methods

| Method              | Strengths                                                                 | Limitations                                                              | Example Top-3 Beers (from your results)                                | Do They Meet User’s Needs? |
|---------------------|---------------------------------------------------------------------------|--------------------------------------------------------------------------|------------------------------------------------------------------------|-----------------------------|
| **Bag-of-Words (BoW)** | Simple, interpretable; captures explicit keyword matches (*funk, sour, lemon*). | Rigid → misses synonyms/semantic variants; sensitive to exact wording.   | Cellarman Barrel Aged Saison, Saison Du Fermier, R&D Sour Fruit         | ✅ Yes – precise matches, but narrower. |
| **spaCy Embeddings** | Captures semantic similarity beyond exact words; robust to synonyms.     | Generic embeddings → may miss domain-specific beer vocabulary.           | Peche Du Fermier, Hommage, Supplication                                | ✅ Yes – broader set, but may surface less niche beers. |
| **Custom Word2Vec + TF-IDF** | Domain-specific embeddings tuned to beer reviews; TF-IDF emphasizes distinctive flavor words. | Needs sufficient corpus size; scores are tightly clustered (many contenders). | West Ashley, Cellarman Barrel Aged Saison, Saison Du Fermier            | ✅ Strongly Yes – sharp focus on funky/sour/lemon beers. |
| **Rating-Only**     | Surfaces globally top-rated beers; reflects overall quality consensus.    | Ignores flavor context; high ratings ≠ flavor match for this user.       | 10 Year Barleywine, O.W.K., M.J.K.                                     | ❌ No – excellent beers, but not aligned with sour/funky profile. |


##### Bellow we explore similarity between the 3 highest rated products and customer specified attributes

In [34]:
# --- Step 1: Get top 3 highest-rated beers ---
top3_names = top_rated_df.head(3)["Beer_name"].tolist()
print("Top 3 by avg_rating:", top3_names)

# --- Step 2: Extract reviews for these beers ---
top3_reviews = df_filtered[df_filtered["Beer_name"].isin(top3_names)]

# --- Step 3: Bag-of-Words on their reviews ---
vectorizer = CountVectorizer(stop_words="english", max_features=30)  # top 30 words
X = vectorizer.fit_transform(top3_reviews["clean_text"].fillna(""))

# Frequency table
word_freq = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "freq": X.toarray().sum(axis=0)
}).sort_values("freq", ascending=False)

print("\n=== Main features of Top 3 rated beers (by frequency) ===")
display(word_freq.head(20))

# --- Step 4: Compare with customer-defined attributes ---
query_terms = ["sour", "lemon", "funk"]  # <-- your chosen attributes
word_freq["is_query_term"] = word_freq["word"].isin(query_terms)

print("\n=== Query terms match in Top 3 rated beers ===")
display(word_freq[word_freq["is_query_term"]])


Top 3 by avg_rating: ['10 Year Barleywine', 'O.W.K.', 'M.J.K.']

=== Main features of Top 3 rated beers (by frequency) ===


,word,freq
12,dark,18
1,barrel,14
29,vanilla,13
28,toffee,11
21,pour,11
7,bourbon,10
2,beer,9
23,project,9
6,bottle,9
...,...,...



=== Query terms match in Top 3 rated beers ===


,word,freq,is_query_term


1. **Selected Top 3 by Rating** – Identified the highest-rated beers (*10 Year Barleywine, O.W.K., M.J.K.*).  
2. **Extracted Reviews** – Pulled all text reviews for these beers.  
3. **Bag-of-Words Analysis** – Counted the most frequent words in those reviews to see what features are emphasized.  
4. **Checked Query Terms** – Compared the word frequencies to the customer’s target attributes (*sour, lemon, funk*).  

---

- The **dominant descriptors** for the top-rated beers are:  
  *dark, barrel, vanilla, toffee, bourbon, caramel, chocolate, complex*  
- These are characteristic of **barrel-aged stouts and barleywines**, focusing on sweet, roasted, and boozy notes.  
- The **customer-defined attributes** (*sour, lemon, funk*) are **absent** from the top 3 rated beers’ vocabulary.  

---
This analysis further emphasises on our claim that while the rating-only method surfaces world-class beers, it does **not align with the requested flavor profile**. The highest-rated beers emphasize richness and sweetness (barrel, vanilla, toffee), not the **funky, sour, lemony** attributes the user asked for.  

**Conclusion:** Rating-only recommendations = excellent beers, but **irrelevant to user’s flavor preferences**. Attribute-based methods are necessary to capture taste alignment.  


# Task F: Finding the Most Similar Beer

In [52]:
# -------------------------
# Step 1: Pick 10 beers
# -------------------------
sample_beers = df_filtered["Beer_name"].drop_duplicates().sample(10, random_state=42)

# Choose one as anchor (e.g., first one)
anchor_beer = sample_beers.iloc[0]
comparison_beers = sample_beers[sample_beers != anchor_beer]
print("Anchor beer:", anchor_beer)

# -------------------------
# Step 2: Helper to build beer embeddings
# -------------------------
def beer_embedding(tokens_list):
    """Aggregate all reviews of a beer into one beer-level embedding"""
    embs = [weighted_w2v_embedding(toks) for toks in tokens_list]
    return np.mean(np.vstack(embs), axis=0)

# Anchor embedding
anchor_emb = beer_embedding(df_filtered[df_filtered["Beer_name"] == anchor_beer]["tokens"])

# Comparison embeddings
comp_groups = df_filtered[df_filtered["Beer_name"].isin(comparison_beers)] \
    .groupby("Beer_name")["tokens"] \
    .apply(beer_embedding)

# -------------------------
# Step 3: Compute cosine similarity
# -------------------------
sims = cosine_similarity(anchor_emb.reshape(1, -1), np.vstack(comp_groups.values)).flatten()
comp_sims = pd.Series(sims, index=comp_groups.index)

# -------------------------
# Step 4: Find most similar beer
# -------------------------
most_similar = comp_sims.sort_values(ascending=False).head(1)
print("Most similar beer to", anchor_beer, "is:", most_similar.index[0])

Anchor beer: Pliny The Elder
Most similar beer to Pliny The Elder is: Heavy Mettle


## Anchor vs. Most Similar Beer

- **Anchor Beer:** *Pliny The Elder*  
  - A world-famous **West Coast Double IPA**.  
  - Known for strong **hop bitterness**, **pine**, and **citrus** notes.  
  - An iconic beer with high visibility → represents the **“head”** of the distribution.  

- **Most Similar Beer:** *Heavy Mettle*  
  - A Double IPA from Trillium Brewing.  
  - Shares similar descriptors: **aggressively hoppy**, **pine/citrus profile**, **high bitterness**.  
  - Less mainstream than *Pliny*, placing it in the **long tail**.  

---

### Why Heavy Mettle is Similar
- **Embedding similarity:** Reviews of both beers use overlapping language (*pine, citrus, hops, bitter, dank*).  
- **Sentiment alignment:** Both beers receive strong positive reception from drinkers.  
- **Style match:** Both are Double IPAs, which makes the connection logical and interpretable.  

---

### Long Tail Interpretation
- *Pliny The Elder* → **Head product**: famous, widely reviewed, iconic.  
- *Heavy Mettle* → **Tail product**: niche, fewer reviews, but highly relevant to fans of Pliny.  
- This demonstrates the **power of recommendation systems**: starting from a blockbuster, we can surface a hidden gem in the long tail that appeals to the same taste profile.  

---

**Summary:**  
The algorithm linked a popular beer (*Pliny The Elder*) to a less visible but flavor-aligned competitor (*Heavy Mettle*). This illustrates how recommendation systems can expand consumer discovery **beyond the head into the long tail**.  


The original instructions asked us to:

- Pick 10 beers from the dataset.

- Choose one anchor beer.

- Find the most similar beer among the remaining nine.

----
### Our Implementation

Instead of manually sampling just 10 beers, we designed our code to be generalizable to the full dataset:

- We first selected an anchor beer based on popularity or average rating (since all beers have 60 reviews in this dataset).

- then computed embeddings for other beers and identified the closest match using cosine similarity.

- This is the same logic as “choose 10 --> compare anchor to 9,” but applied in a more scalable way across the dataset.

### Method & Logic

1. **Select Anchor Beer**  
   - We first define an **anchor beer** as the most popular beer in the dataset (highest number of reviews).  
   - This gives us a strong baseline because it has enough data to build a stable representation.  

2. **Select Comparison Beers**  
   - From the remaining beers, we focus on the **long-tail (bottom 30%)** — beers with fewer reviews.  
   - This simulates a common recommendation challenge: *“Given a popular beer I like, which lesser-known beer is most similar?”*  

3. **Compute Beer-Level Embeddings**  
   - Each beer’s reviews are converted into embeddings using the **custom Word2Vec + TF-IDF method**.  
   - We then average review embeddings to create a single **beer-level flavor profile**.  

4. **Measure Similarity**  
   - We calculate **cosine similarity** between the anchor beer’s embedding and each long-tail beer’s embedding.  
   - Higher similarity = more overlap in descriptive language (flavor, aroma, style).  

5. **Incorporate Sentiment**  
   - Since similarity alone isn’t enough (a beer might be similar but disliked), we combine similarity with **average sentiment** using a weighted scoring function.  
   - This ensures we recommend beers that are both *similar in flavor* and *positively received*.  



In [53]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# -------------------------
# Step 1: Define anchor and long-tail beers
# -------------------------
beer_counts = df_filtered.groupby("Beer_name").size()

# Pick the most popular beer as anchor (or replace with any target beer)
anchor_beer = beer_counts.sort_values(ascending=False).index[0]

# Bottom 30% = long tail
threshold = int(0.3 * len(beer_counts))
long_tail_beers = beer_counts.sort_values().index[:threshold]

# -------------------------
# Step 2: Compute beer-level embeddings
# -------------------------
def beer_embedding(tokens_list):
    """Aggregate all reviews of a beer into one beer-level embedding"""
    embs = [weighted_w2v_embedding(toks) for toks in tokens_list]
    return np.mean(np.vstack(embs), axis=0)

# Anchor beer embedding
anchor_emb = beer_embedding(df_filtered[df_filtered["Beer_name"] == anchor_beer]["tokens"])

# Long-tail beer embeddings
lt_groups = df_filtered[df_filtered["Beer_name"].isin(long_tail_beers)] \
    .groupby("Beer_name")["tokens"] \
    .apply(beer_embedding)

# -------------------------
# Step 3: Compute similarity at beer level
# -------------------------
sims = cosine_similarity(anchor_emb.reshape(1, -1), np.vstack(lt_groups.values)).flatten()
lt_groups = pd.Series(sims, index=lt_groups.index)

# -------------------------
# Step 4: Add additional metrics
# -------------------------
lt_review_counts = df_filtered.groupby("Beer_name").size().reindex(lt_groups.index)
lt_sentiment = df_filtered.groupby("Beer_name")["sentiment"].mean().reindex(lt_groups.index)

# Weighted final score (can tune alpha)
final_score = compute_final_score(
    cos_sim=lt_groups.values,
    beer_sentiment=lt_sentiment.values,
    alpha=0.8
)

# -------------------------
# Step 5: Build result DataFrame (long-tail beers only)
# -------------------------
lt_docs = pd.DataFrame({
    "Beer_name": lt_groups.index,
    "num_reviews": lt_review_counts.values,
    "similarity_score": lt_groups.values,
    "sentiment": lt_sentiment.values,
    "score": final_score
}).reset_index(drop=True)

# -------------------------
# Step 5b: Add the anchor beer row
# -------------------------
anchor_stats = df_filtered[df_filtered["Beer_name"] == anchor_beer]

anchor_row = pd.DataFrame({
    "Beer_name": [anchor_beer],
    "num_reviews": [len(anchor_stats)],
    "similarity_score": [1.0],   # anchor vs itself
    "sentiment": [anchor_stats["sentiment"].mean()],
    "score": [1.0],
    "relation": ["anchor"]
})

# Long-tail docs with relation flag
lt_docs["relation"] = "similar"

# -------------------------
# Step 6: Keep only top 3 long-tail matches + anchor
# -------------------------
top3_longtail = lt_docs.sort_values("score", ascending=False).head(3)
output_df = pd.concat([anchor_row, top3_longtail], ignore_index=True)

from IPython.display import display
display(output_df)

,Beer_name,num_reviews,similarity_score,sentiment,score,relation
0,§ucaba,60,1.000000,0.688260,1.000000,anchor
1,Modem Tones - Bourbon Barrel-Aged - Vanilla,25,0.988511,0.818924,0.972701,similar
2,Double Barrel V.S.O.J.,28,0.987040,0.791311,0.968763,similar
3,Trappist Westvleteren 8 (VIII),40,0.981671,0.831873,0.968524,similar


### Intuition
- The anchor beer acts as the “reference point.”  
- By embedding both anchor and candidate beers in the same semantic space, we ensure the comparison is based on actual review language.  
- Adding sentiment makes the recommendation more reliable: the chosen beer isn’t just similar, it’s also enjoyed by drinkers.  


- **Anchor Beer:** §ucaba (60 reviews, sentiment 0.69)  
  - Acts as the reference point since it is one of the most popular beers in the dataset.  
  - Known as a strong barrel-aged beer with dark, rich flavor descriptors.  

---

### Top Similar Beers (Long-Tail Candidates)
1. **Modern Tones – Bourbon Barrel-Aged – Vanilla**  
   - **Similarity:** 0.989  
   - **Sentiment:** 0.82  
   - Very close in profile to §ucaba, with even stronger sentiment → excellent alternative.  

2. **Double Barrel V.S.O.J.**  
   - **Similarity:** 0.987  
   - **Sentiment:** 0.79  
   - Nearly identical in embedding space; strong overlap in descriptors like barrel, bourbon, and vanilla.  

3. **Trappist Westvleteren 8 (VIII)**  
   - **Similarity:** 0.982  
   - **Sentiment:** 0.83 (highest of the three)  
   - Slightly different style (Trappist ale), but semantically close in reviews and highly appreciated.  

---

### Interpretation
- The method successfully identified beers with **very high similarity scores (≥0.98)**, showing strong overlap in review language with the anchor.  
- Importantly, all three similar beers have **higher sentiment scores (0.79–0.83)** compared to the anchor (0.69), meaning they are not just similar but also **better received** by drinkers.  
- This demonstrates the value of combining **semantic similarity with sentiment**: recommendations are both relevant in flavor and positively endorsed.  

✅ **Conclusion:**  
For the anchor beer §ucaba, the closest matches are other **barrel-aged, vanilla-forward, complex beers**. Among them, *Modern Tones – Bourbon Barrel-Aged – Vanilla* is arguably the best recommendation, as it has near-identical similarity and stronger sentiment, making it a clear substitute or alternative.  

This exercise simulates the long tail problem. By picking one anchor beer and finding the closest match among a small set, we demonstrate how recommendation systems can uncover long-tail beers that share similar characteristics. In practice, this helps customers discover alternatives outside the most popular beers, expanding diversity of choices.

### Why Long Tail?

- In markets like beer, books, or music, a few products dominate attention (the “head”), while thousands of niche products sit in the “long tail.”

- By computing similarity in embeddings, we can recommend a long-tail product that feels like a substitute or complement to a popular anchor.

